# Creating a Portuguese Dictionary

This mini Python project aims at building a PT dictionary using webscraping and OOP.

## Preparing the list of words to search in the dictionary

corpus = http://www.ufrgs.br/textecc/porlexbras/corpop/lista.php

CorPop: a reference corpus of popular written Portuguese from Brazil

CorPop is a corpus of written popular Brazilian Portuguese, compiled from selected texts based on the average literacy level of the country's readers. _(description taken from the corpus website)_

In [1]:
import unicodedata

In [2]:
path = "C:/Users/joaop/Documents/oop/"
import re
with open(path + 'corprop_lemas.txt', 'r', encoding='iso-8859-1') as f: #reading the list of words file
    
    #LÊ O TEXTO
    words_corpus = f.read()
    new_word_corpus = ''.join(ch for ch in unicodedata.normalize('NFKD', words_corpus) #replacing the PT special characters
    if not unicodedata.combining(ch))

In [3]:
list_words_corpus = new_word_corpus.split("\n") #transforming the string into a list (each item is a word)
list_words_corpus[100:109]

['admitir',
 'adocao',
 'adoecer',
 'adolescencia',
 'adolescente',
 'adoracao',
 'adorar',
 'adormecer',
 'adormecido']

In [4]:
# creating the URLs based on the corpus list
pages = []
for word in list_words_corpus[:100]: #trying out w/ 100 just to check if it works
    url = f"http://www.dicio.com.br/{word}/"
    pages.append(url)

print(len(pages))

100


## Preparing and doing the webscraping

In [5]:
import requests
from bs4 import BeautifulSoup as bs
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'})

In [32]:
def get_info_from_dict(page): 
    #function that retrieves all information of a word at once: thw word, its meaning
    # its definition and an example in a sentence
    import re
    web_response = requests.get(page, headers=HEADERS)
    soup = bs(web_response.text, "html.parser")
    definition_html = soup.find(class_ = "significado textonovo")
    definition_str = str(definition_html)
    definition_str = re.sub("\<\/span\>", "", definition_str)
    definition_str = re.sub("\<\span\>", "", definition_str)
    definition_str= re.sub("\<span", "", definition_str)
    definition_final = definition_str.split(">")[3]
    example_html = soup.find(class_ = "frase")
    example_str = str(example_html)
    example_main = re.findall("(?<=\<span\>).*(?=\<br\/>)", example_str)
    example_main = str(example_main)
    example_final = re.sub("\[", "", example_main)
    example_final2 = re.sub("\]", "", example_final)
    class_html = soup.find(class_ = "cl").text
    # class_name = re.sub("\<span class=\"cl\"\>", " ", class_html)
    # class_name = re.sub("\<\/span\>", " ", class_html)
    word_title = soup.find("h1").text
    return word_title, definition_final, example_final2, class_html

In [23]:
word_infos = []
for page in pages:
    c, d, e, f = get_info_from_dict(page) # looping the function to get the information for all words in the list
    word_infos.append([c, d, e, f]) 

In [24]:
len(word_infos) #checking

100

## Creating the dictionary class

In [25]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [26]:
class Entry: 
    #entry = all information about a word
    def __init__(self, word): #the word
        self.word = word
        # print(f"{word} added to the Dictionary")

    def create_class(self, classes):   #the grammatical class 
        self.classes = classes
        # print(f"{transcription} added to the Dictionary")
    
    def create_meaning(self, meaning): #the definition
        self.meaning = meaning
        # print(f"{meaning} added to the Dictionary")

    def create_example(self, example): #the example
        self.example = example 
        # print(f"{example} added to the Dictionary") 

In [43]:
class Dictionary(Entry):
    # methods here all related only to the dictionary query
    def __init__(self, palavras):
        self.palavras = palavras

    def definition_search(self): #get only the semantic meaning
        word_query = str(input("Write the word you want to know the meaning"))
        for obj_word in self.palavras:
            if word_query == obj_word.word: 
                word_bold = color.RED + f"{obj_word.word}" + color.END
                meaning_bold = color.RED +f"{obj_word.meaning}"+ color.END
                print(f"The meaning of {word_bold} is {meaning_bold}")
                break
        else:
            print("Word not found")
    
    def get_full_entry(self): #get the transcription, semantic meaning and example (full entry)
        full_entry_query = str(input("Write the word you want to know the full entry"))
        for obj_word in self.palavras:
            if full_entry_query == obj_word.word:
                print(f"{color.BOLD}Word{color.END}: {obj_word.word}\n{color.BOLD}Class{color.END}: {obj_word.classes}\n{color.BOLD}Meaning{color.END}: {obj_word.meaning}\n{color.BOLD}Example:{color.END} {obj_word.example}") 
                break
                
        else:
            print("Word not found")  
    
    def get_full_dict(self): #get full entry of all words in the dictionary
        words_sorted = sorted(self.palavras, key=lambda palavra: palavra.word)
        for obj_word in words_sorted:
            print(f"Word: {obj_word.word}\nClass: {obj_word.classes}\nMeaning: {obj_word.meaning}\nExample: {obj_word.example}")
            print("\n==========")

    def delete_full_entry(self): # delete full entry of a word in the dictionary
        entry_query = str(input("Write the word whose full entry you want to delete"))
        for obj_word in self.palavras:
            if entry_query ==  obj_word.word:
                self.palavras.remove(obj_word)
                print(f"Entry of word *{obj_word.word}* deleted successfully")
                break
        else:
            print("Word not found. Try again")

## Putting it to work

In [44]:
list_entry = []
for word_info in word_infos:
    word = Entry(word_info[0])
    word.create_meaning(word_info[1])
    word.create_example(word_info[2])
    word.create_class(word_info[3])
    # print(word_info)
    list_entry.append(word)

In [40]:
for entry in list_entry[90:99]:
    print(entry.word) #checking

adivinhar
adivinho
administração
administrador
administrar
administrativo
admiração
admirando
admirar


In [45]:
pt_dict = Dictionary(list_entry)

In [46]:
pt_dict.definition_search() # do a query for the meaning of a word

The meaning of acabar is Levar a seu termo, ao fim; perfazer, concluir, terminar: acabar uma tarefa para o trabalho; nosso casamento se acabou com o tempo. class="cl"


In [42]:
pt_dict.get_full_entry() # do a query for the full entry of a word

Word: acabar
Class: verbo bitransitivo, intransitivo e pronominal
Meaning: Levar a seu termo, ao fim; perfazer, concluir, terminar: acabar uma tarefa para o trabalho; nosso casamento se acabou com o tempo. class="cl"
Example: 'É melhor calar-se e deixar que as pessoas pensem que você é um idiota do quer falar e acabar com a dúvida.'


In [17]:
pt_dict.get_full_dict() #get the full dictionary 
# Attention: if you put a lot of entries in it, your IDE probably will show a portion of it

Word: aba
Class: substantivo feminino
Meaning: Parte destacável de algo que o completa; extremidade.
Example: 

Word: abafar
Class: verbo transitivo
Meaning: Impedir a respiração; sufocar, asfixiar.
Example: 'Eles pensavam poder abafar e vencer a verdade, que é sempre vitoriosa, ignorando que a própria essência da verdade é que quanto mais quisermos comprimi-la mais ela cresce e se eleva.'

Word: abaixar
Class: verbo transitivo
Meaning: Fazer descer; pôr em nível mais baixo.
Example: 'Nenhum exercício é melhor para o coração humano do que se abaixar e levantar alguém.'

Word: abaixo
Class: advérbio
Meaning: Embaixo; em sentido menos elevado; num lugar mais baixo; num ponto inferior a: descemos rua abaixo; a padaria está logo abaixo.
Example: 'Posso perdoar a força bruta, mas a razão bruta é uma coisa irracional. É bater abaixo da linha do intelecto.'

Word: abajur
Class: substantivo masculino
Meaning: Peça que recobre lâmpadas, candeeiros etc., para amortecer a intensidade da luz. clas

In [18]:
pt_dict.delete_full_entry() # delete an entry
# tip test deleting the first word so it's going to be easier to check if it's working
pt_dict.get_full_dict() #checking if it worked


Entry of word *aba* deleted successfully
Word: abafar
Class: verbo transitivo
Meaning: Impedir a respiração; sufocar, asfixiar.
Example: 'Eles pensavam poder abafar e vencer a verdade, que é sempre vitoriosa, ignorando que a própria essência da verdade é que quanto mais quisermos comprimi-la mais ela cresce e se eleva.'

Word: abaixar
Class: verbo transitivo
Meaning: Fazer descer; pôr em nível mais baixo.
Example: 'Nenhum exercício é melhor para o coração humano do que se abaixar e levantar alguém.'

Word: abaixo
Class: advérbio
Meaning: Embaixo; em sentido menos elevado; num lugar mais baixo; num ponto inferior a: descemos rua abaixo; a padaria está logo abaixo.
Example: 'Posso perdoar a força bruta, mas a razão bruta é uma coisa irracional. É bater abaixo da linha do intelecto.'

Word: abajur
Class: substantivo masculino
Meaning: Peça que recobre lâmpadas, candeeiros etc., para amortecer a intensidade da luz. class="etim"
Example: 

Word: abalada
Class: substantivo feminino
Meaning: 